In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW
class Net(nn.Module):
    def __init__(self, model_name):
        super(Net, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, 1)
        
    def forward(self, ids, mask):        
        
        out = self.model(input_ids=ids,
                         attention_mask=mask,
                         output_hidden_states=False)
            
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

class ReviewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = self.df['comments'].values
        self.y = df['review_scores_rating'].values
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        y = self.y[index]
        
        inputs_tokenizer = self.tokenizer(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
        inputs_ids = inputs_tokenizer['input_ids']
        inputs_mask = inputs_tokenizer['attention_mask']
           
        return {
            'inputs_ids': torch.tensor(inputs_ids, dtype=torch.long),
            'inputs_mask': torch.tensor(inputs_mask, dtype=torch.long),
            'value': torch.tensor(y, dtype=torch.float)
            
        }
model_name = 'bert-base-multilingual-cased'
net = Net(model_name)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
net.load_state_dict(torch.load('Loss-Fold-0.bin'))

<All keys matched successfully>

In [52]:
import pandas as pd
df = pd.read_csv('test.csv')
df['comments'] = df['comments'].astype(str)

df['review_scores_rating'] = df['review_scores_rating'].astype(float)
df = df[~df['review_scores_rating'].isna()]

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [54]:
from tqdm.notebook import tqdm
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
train_dataset = ReviewsDataset(df, tokenizer=tokenizer, max_length=256)
train_loader = DataLoader(train_dataset, batch_size=1, num_workers=2, shuffle=True, pin_memory=True, drop_last=True)

@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    model.to(device)
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    for step, data in enumerate(dataloader):
        ids = data['inputs_ids'].to(device, dtype = torch.long)
        mask = data['inputs_mask'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
    PREDS = np.concatenate(PREDS)    
    return PREDS


In [55]:
import numpy as np
def inference(model, dataloader, device):
    final_preds = []
    model.eval()
    preds = valid_fn(model, dataloader, device)
    final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds
result = inference(net, train_loader, 'cuda')

In [56]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['review_scores_rating'],result)

0.10480221646658998

* O modelo apresenta um baixo mse demonstrando um bom fit com os dados reais, porém novamente o dado tem uma baixa quantidade de avaliações 'ruins' então o modelo vai "tender" as avaliações altas.